<a href="https://colab.research.google.com/github/Madhavi-1234/LoRA_LLM_Practice/blob/colab_notebook/LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q transformers datasets accelerate peft bitsandbytes torch huggingface_hub

In [4]:
from huggingface_hub import login
login(token="your huggigface token")

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name = "facebook/opt-1.3b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
# Quick Inference Check
prompt = "Hello AI, how are you today?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Hello AI, how are you today?
I'm good, how are you?                                         


In [7]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config=  LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj"],
    bias= "none"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,317,330,944 || trainable%: 0.1194


In [8]:
from datasets import load_dataset

dataset = load_dataset("Abirate/english_quotes")

def tokenize_function(example):
    return tokenizer(example["quote"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)


README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./lora_output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=50,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator
)

trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
50,2.467900
100,2.487700
150,2.318600
200,2.386200
250,2.444300
300,2.420100
350,2.403500
400,2.383700
450,2.300000
500,2.328000


TrainOutput(global_step=942, training_loss=2.3614784613283324, metrics={'train_runtime': 376.3005, 'train_samples_per_second': 19.995, 'train_steps_per_second': 2.503, 'total_flos': 6992916765474816.0, 'train_loss': 2.3614784613283324, 'epoch': 3.0})

In [13]:
for n, p in model.named_parameters():
    if "lora" in n:
        print(n, p.abs().mean().item())


base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_A.default.weight 0.011505035683512688
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_B.default.weight 0.0030241655185818672
base_model.model.model.decoder.layers.0.self_attn.q_proj.lora_A.default.weight 0.01174049824476242
base_model.model.model.decoder.layers.0.self_attn.q_proj.lora_B.default.weight 0.003883073106408119
base_model.model.model.decoder.layers.1.self_attn.v_proj.lora_A.default.weight 0.011416061781346798
base_model.model.model.decoder.layers.1.self_attn.v_proj.lora_B.default.weight 0.0024560349993407726
base_model.model.model.decoder.layers.1.self_attn.q_proj.lora_A.default.weight 0.011621296405792236
base_model.model.model.decoder.layers.1.self_attn.q_proj.lora_B.default.weight 0.0029380694031715393
base_model.model.model.decoder.layers.2.self_attn.v_proj.lora_A.default.weight 0.011586512438952923
base_model.model.model.decoder.layers.2.self_attn.v_proj.lora_B.default.weight 0.00238523026928305

In [11]:
model.save_pretrained("/content/drive/MyDrive/lora_adapter")


In [16]:
prompt = "Write a short poem about AI:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,            # sampling instead of greedy
    temperature=0.8,           # controls creativity
    top_p=0.9,                 # nucleus sampling
    repetition_penalty=1.2,    # prevent loops
    eos_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Write a short poem about AI: "I'm an artificial intelligence, I can't get angry"

The best way to learn how to write a decent sentence is through reading good writing. And the only reason why we've come up with all these rules is because they were learned that way. It's not by accident." - Mark TwainIf you want to be successful in life, you have to read books, so don't complain when people say you're too smart for them." - Charles DickensSo if you find yourself being criticized and don't know what to say back, just let it go like this:"You should stop whining about everything."- Mark TwainRead my book on how to make friends everywhere (and be funny at the same time) here.http://www.goodreads.com/book?id=10889401Read some of my other books or follow me on Twitter @the_shoe_manFor more information about how AI works check out my new video series on YouTube http://youtu


In [17]:
from google.colab import files
uploaded = files.upload()  # Select the downloaded LoRA.ipynb



/

Saving LoRA.ipynb to LoRA (1).ipynb


()

In [22]:
import json

notebook_file = "LoRA.ipynb"

with open(notebook_file, "r", encoding="utf-8") as f:
    nb = json.load(f)

# Remove all 'metadata.widgets' from all cells
for cell in nb['cells']:
    if 'metadata' in cell and 'widgets' in cell['metadata']:
        del cell['metadata']['widgets']

# Save cleaned notebook
with open(notebook_file, "w", encoding="utf-8") as f:
    json.dump(nb, f, indent=1)


In [20]:
!jupyter nbconvert --clear-output --inplace LoRA.ipynb


[NbConvertApp] Converting notebook LoRA.ipynb to notebook
[NbConvertApp] Writing 138100 bytes to LoRA.ipynb


In [23]:
!mv /content/LoRA.ipynb /content/LoRA_LLM_Practice/
%cd /content/LoRA_LLM_Practice
!git add LoRA.ipynb
!git commit -m "Clean LoRA notebook: removed widgets + cleared outputs"
!git push origin colab_notebook


mv: cannot stat '/content/LoRA.ipynb': No such file or directory
/content/LoRA_LLM_Practice
[main 29fbc2b] Clean LoRA notebook: removed widgets + cleared outputs
 1 file changed, 4535 insertions(+)
 create mode 100644 LoRA.ipynb
error: src refspec colab_notebook does not match any
error: failed to push some refs to 'https://github.com/Madhavi-1234/LoRA_LLM_Practice.git'
